In [ ]:
import pandas as pd

energy_data = pd.read_csv("Extra//energy.csv")
energy_data['timestamp'] = pd.to_datetime(energy_data['timestamp'])
energy_data.set_index('timestamp', inplace=True)
resampled_energy = energy_data.resample("5s").mean()
resampled_energy = resampled_energy.fillna(method='ffill')
resampled_energy = resampled_energy.fillna(method='bfill')

env_data = pd.read_csv("Extra//environment.csv")
env_data['timestamp'] = pd.to_datetime(env_data['timestamp'])
env_data.set_index('timestamp', inplace=True)
resampled_env = env_data.resample("5s").mean()
resampled_env = resampled_env.fillna(method='ffill')
resampled_env = resampled_env.fillna(method='bfill')

resampled_energy['reactive_power'] = resampled_energy[["Reactive Power A average [kVAr]","Reactive Power B average [kVAr]","Reactive Power C average [kVAr]"]].mean(axis=1)
resampled_energy['thdi'] = resampled_energy[["THDI A average [%]","THDI B average [%]","THDI C average [%]"]].mean(axis=1)
resampled_energy['thdu'] = resampled_energy[["THDU A average [%]","THDU B average [%]","THDU C average [%]"]].mean(axis=1)
resampled_energy['current'] = resampled_energy[["Current A average [A]","Current B average [A]","Current C average [A]"]].mean(axis=1)
resampled_energy['voltage'] = resampled_energy[["Voltage A average [V]","Voltage B average [V]","Voltage C average [V]"]].mean(axis=1)
resampled_energy['power_factor'] = resampled_energy[["Power Factor A average","Power Factor B average","Power Factor C average"]].mean(axis=1)
useful_data = resampled_energy.join(resampled_env)
useful_data = useful_data[["reactive_power","power_factor","current","voltage","thdu","thdi","Xacc","yaw","pitch"]]
useful_data = useful_data.dropna()
display(useful_data)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_data = scaler.fit_transform(useful_data[useful_data.columns])

In [ ]:
# find the optimal number of clusters
# when uncommenting this, use the model (from below) as:
# model = hmm.GaussianHMM(n_components = n_states_range[np.argmax(mdl_scores)], covariance_type='diag')

"""from hmmlearn import hmm
import numpy as np
#np.random.seed(87)
import matplotlib.pyplot as plt

def mdl_score(model, data):
    n_features = data.shape[1]
    n_states = model.n_components
    
    n_transition_params = n_states * (n_states - 1)
    n_emission_params = n_states * n_features
    n_initial_state_params = n_states - 1
    n_params = n_transition_params + n_emission_params + n_initial_state_params
    
    adjusted_bic = model.score(data) - np.square(n_params) * np.log(data.shape[0])
    return adjusted_bic

n_states_range = range(4, 9)
mdl_scores = []

for n_states in n_states_range:
    mdl_scores_n_states = []

    for run in range(10):
        model = hmm.GaussianHMM(n_components=n_states, covariance_type='diag')
        model.fit(scaled_data)
        mdl = mdl_score(model, scaled_data)
        mdl_scores_n_states.append(mdl)

    avg_mdl = np.mean(mdl_scores_n_states)
    print(n_states)
    print(avg_mdl)
    mdl_scores.append(avg_mdl)

plt.figure(figsize=(8, 6))
plt.plot(n_states_range, mdl_scores, marker='o', linestyle='-', linewidth=2)
plt.xlabel("Number of hidden states")
plt.ylabel("Average ABIC score")
plt.grid()
plt.show()"""

In [ ]:
from hmmlearn import hmm
import numpy as np
np.random.seed(33)

#model = hmm.GaussianHMM(n_components = n_states_range[np.argmax(mdl_scores)], covariance_type='diag')
model = hmm.GaussianHMM(n_components = 5, covariance_type='diag')
model.fit(scaled_data)
hidden_states = model.predict(scaled_data)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
normalized_plot_data = pd.DataFrame(scaler.fit_transform(useful_data.values), columns=useful_data.columns, index=useful_data.index)

normalized_plot_data = normalized_plot_data.assign(states = hidden_states)
normalized_plot_data.insert(loc=0, column='Date', value=pd.to_datetime(normalized_plot_data.index))
normalized_plot_data['modes'] = normalized_plot_data['states'].map({0:'Offline', 1: 'InMotion', 2: 'Mode2', 3: 'Online', 4:'Mode1'})
color_map = {"Offline": "black", "InMotion": "Green", "Online": "white", "Mode1": "yellow", "Mode2" : "magenta"}

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

plot_data = normalized_plot_data.loc['2022-11-16 13:50:00':'2022-11-16 16:10:00']

fig = px.line(plot_data, x='Date', y='current')
fig.update_traces(line=dict(color='black'))
fig.update_layout(xaxis_title="Time", yaxis_title="Current average [A]", xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))

#start background
start_mode = str(plot_data.iloc[0]["modes"])
start_date = str(plot_data.iloc[0]["Date"])

for index, row in plot_data.iterrows():
    current_mode = row["modes"]
    if current_mode != start_mode:
        fig.add_vrect(x0=start_date, x1=str(row["Date"]), fillcolor=color_map[start_mode], opacity=0.5)
        start_mode = row["modes"]
        start_date = str(row["Date"])

fig.add_vrect(x0=start_date, x1=str(plot_data.iloc[-1]["Date"]), fillcolor=color_map[start_mode], opacity=0.5)
for state, color in color_map.items():
    fig.add_trace(go.Scatter(x=[None], y=[None],
                             mode='markers',
                             marker=dict(size=10, color=color),
                             name=state))

#end background
fig.update_layout(height=600,width=1000)
fig.write_image("clusters_2.png")
fig.show()

In [ ]:
#get anomalies
log_probability = model._compute_log_likelihood(scaled_data)
likelihoods = np.sum(np.exp(log_probability), axis=1)
threshold = np.percentile(likelihoods, 1)
anomalies = np.where(likelihoods < threshold)[0]
#write anomalies to csv
normalized_plot_data.reset_index(inplace=True)
normalized_plot_data['Anomaly'] = 'No'
normalized_plot_data.loc[anomalies,'Anomaly'] = 'Yes'
normalized_plot_data.set_index('timestamp', inplace=True)
normalized_plot_data[['Date', 'modes', 'Anomaly']].to_csv('hmm_anomalies.csv', index=False)

In [ ]:
for anomaly in anomalies[:15]:
    df = normalized_plot_data.loc[str(useful_data.iloc[anomaly-100].name):str(useful_data.iloc[anomaly+100].name)]
    line_fig = px.line(df, x = 'Date', y = useful_data.columns)
    line_fig.update_traces(line=dict(color = 'black'), visible='legendonly')
    fig = go.Figure(data=line_fig.data)
    fig.update_layout(xaxis_title='Time', yaxis_title='MinMaxed Data', xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))

    #start background
    start_mode = str(df.iloc[0]["modes"])
    start_date = str(df.iloc[0]["Date"])

    for index, row in df.iterrows():
        current_mode = row["modes"]
        if current_mode != start_mode:
            fig.add_vrect(x0=start_date, x1=str(row["Date"]), fillcolor=color_map[start_mode], opacity=0.5)
            start_mode = row["modes"]
            start_date = str(row["Date"])

    fig.add_vrect(x0=start_date, x1=str(df.iloc[-1]["Date"]), fillcolor=color_map[start_mode], opacity=0.5)
    #end background and highlight anomaly
    fig.add_vrect(x0=str(useful_data.iloc[anomaly-1].name), x1=str(useful_data.iloc[anomaly+1].name), fillcolor="red", opacity=0.5)

    fig.show()